In [1]:
import typing

import pandas as pd

import config
import src

In [2]:
CFG = config.Config()

In [15]:
dataset: pd.DataFrame = pd.read_parquet(CFG.final_data_files["user_content"]).reset_index("userId")
dataset

,userId,content,createdAt
id,,,
66252255616a4cb7061ee32c,661d1639b8beabb58229451b,#UkraineKrieg: Beunruhigende Entwicklungen in ...,2024-04-21 14:27:33.315
66253e74616a4cb7061ee7cc,661d1639b8beabb58229451b,"Bedauerlich, dass gewaltsame Konflikte weiterh...",2024-04-21 16:27:32.919
66255134616a4cb7061eeaf4,661d1639b8beabb58229451b,Gewalt im Nahen Osten & Ukraine nicht isoliert...,2024-04-21 17:47:32.762
6627d8efa6b56b19b4300ae4,661d1639b8beabb58229451b,"Unterstützen wir unsere Verbündeten, respektie...",2024-04-23 15:51:11.067
6628b08fa6b56b19b4301867,661d1639b8beabb58229451b,Geschlossene Tür für Unsicherheit: US-Repräsen...,2024-04-24 07:11:11.797
...,...,...,...
662a070537d6395f42ca7bfd,662a070537d6395f42ca7bf9,Kleiner vermieter Junge noch nicht gefunden,2024-04-25 07:32:21.584
662a12fa37d6395f42ca87fe,662a12fa37d6395f42ca87fa,Lok Leipzig holt Jochen Seitz als Trainer,2024-04-25 08:23:22.984
662a30f137d6395f42ca99c6,662a30f137d6395f42ca99c2,Russland droht der USA,2024-04-25 10:31:13.258


In [4]:
MODEL: str = "llama3:70b-instruct-q6_K"
CFG.prompt_files.keys()

dict_keys(['political_negativity', 'political_ideology'])

In [5]:
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["content"], MODEL)
    )
    for label, path in CFG.prompt_files.items()
}

classifying: political_ideology: 100%|██████████| 592/592 [06:25<00:00,  1.53it/s]


In [16]:
dataset_w_pred = dataset.copy()

for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    dataset_w_pred = dataset_w_pred.join(preds)
    

political_negativity
political/negative        420
not political/negative    172
Name: count, dtype: int64
------------------------------------------
political_ideology
neutral         276
liberal         272
conservative     44
Name: count, dtype: int64
------------------------------------------


In [17]:
dataset_w_pred

,userId,content,createdAt,political_negativity,political_ideology
id,,,,,
66252255616a4cb7061ee32c,661d1639b8beabb58229451b,#UkraineKrieg: Beunruhigende Entwicklungen in ...,2024-04-21 14:27:33.315,political/negative,neutral
66253e74616a4cb7061ee7cc,661d1639b8beabb58229451b,"Bedauerlich, dass gewaltsame Konflikte weiterh...",2024-04-21 16:27:32.919,political/negative,liberal
66255134616a4cb7061eeaf4,661d1639b8beabb58229451b,Gewalt im Nahen Osten & Ukraine nicht isoliert...,2024-04-21 17:47:32.762,political/negative,liberal
6627d8efa6b56b19b4300ae4,661d1639b8beabb58229451b,"Unterstützen wir unsere Verbündeten, respektie...",2024-04-23 15:51:11.067,political/negative,liberal
6628b08fa6b56b19b4301867,661d1639b8beabb58229451b,Geschlossene Tür für Unsicherheit: US-Repräsen...,2024-04-24 07:11:11.797,political/negative,liberal
...,...,...,...,...,...
662a070537d6395f42ca7bfd,662a070537d6395f42ca7bf9,Kleiner vermieter Junge noch nicht gefunden,2024-04-25 07:32:21.584,not political/negative,neutral
662a12fa37d6395f42ca87fe,662a12fa37d6395f42ca87fa,Lok Leipzig holt Jochen Seitz als Trainer,2024-04-25 08:23:22.984,not political/negative,neutral
662a30f137d6395f42ca99c6,662a30f137d6395f42ca99c2,Russland droht der USA,2024-04-25 10:31:13.258,political/negative,neutral
